<a href="https://colab.research.google.com/github/DivyaMeenaSundaram/Agentic-AI/blob/main/Agentic_AI_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Current Weather: https://wttr.in

In [1]:
!pip install -q langchain langchainhub langchain-google-genai langchain-community google-generativeai numexpr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


In [3]:
import json, requests
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory


llm = ChatGoogleGenerativeAI(
  model="gemini-2.0-flash-lite",
  temperature=0, max_tokens=500,
  google_api_key=userdata.get('Divya_Key')
)


@tool
def fetch_weather(location: str) -> str:
  """
  Fetches current weather and forecast for a given location.
  Example: fetch_weather(location="London")
  """
  url = f"https://wttr.in/{location}?format=j1"
  resp = requests.get(url, headers={"User-Agent": "langchain-tool"})
  if resp.status_code != 200:
      return f"Error: status {resp.status_code}"
  data = resp.text
  return data
# load any built-in tools and add our custom tool
builtin = load_tools(["llm-math"], llm=llm)
tools = [fetch_weather] + builtin
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


# initialize the agent
agent = initialize_agent(tools, llm,
  agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
  verbose=False, memory=memory,
  handle_parsing_errors=True
)


print("Agent initialized with tools:", [t.name for t in tools])
CUSTOM_INSTRUCTION = 'I prefer SI units and concise responses'

Agent initialized with tools: ['fetch_weather', 'Calculator']


/tmp/ipython-input-3-168022675.py:31: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
/tmp/ipython-input-3-168022675.py:35: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(tools, llm,


In [4]:
prompt = "how is it in bangalore tomorrow?"
response = agent.invoke({"input": CUSTOM_INSTRUCTION + '. ' + prompt})
print(f"Agent: {response['output']}")

Agent: The forecast for Bangalore tomorrow is partly cloudy with a high of 29°C and a low of 20°C. There is a chance of rain in the evening.
